# Saving and Loading a Keras Neural Network
Complex neural networks will take a long time to fit/train. It is helpful to be able to save these neural networks so that they can be reloaded later. A reloaded neural network won't require retraining. Keras provides three formats for neural network saving.
 - **YAML** - Stores the neural network structure(no weights) in YAML file
 - **JSON** - Stores the neural network structure(no weights) in JSON file
 - **HDF5** - Stores the neural network structure(with weights) in HDF5 File. Do not confuse HDF5 with HDFS. They are different. We do not use HDFS in this class.
 
 Usually we want to save in HDF5

In [5]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

save_path = "./datasets"

df = pd.read_csv('https://data.heatonresearch.com/data/t81-558/auto-mpg.csv',
                na_values = ['NA', '?'])
cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# Pandas to numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim = x.shape[1], activation = 'relu')) # hidden 1
model.add(Dense(10, activation = 'relu')) # Hidden 2
model.add(Dense(1)) # output

model.compile(loss = 'mean_squared_error', optimizer = 'adam')

model.fit(x, y, verbose = 2, epochs = 100)

# Predict
pred = model.predict(x)

# Measure RMSE error. RMSE is common for regression
score = np.sqrt(metrics.mean_squared_error(pred, y))

print('Before save score (RMSE) : ',score)

#save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(save_path, 'network.json'), 'w') as json_file:
    json_file.write(model_json)
    
#save neural network structure to YAML (no weights)
model_yaml = model.to_yaml()
with open(os.path.join(save_path, 'network.yaml'), 'w') as yaml_file:
    yaml_file.write(model_yaml)
    
#save neural network structure to HDF5 (with weights)
model.save(os.path.join(save_path, 'network.h5'))


Epoch 1/100
 - 0s - loss: 38307.5818
Epoch 2/100
 - 0s - loss: 2649.4344
Epoch 3/100
 - 0s - loss: 1739.7579
Epoch 4/100
 - 0s - loss: 453.7089
Epoch 5/100
 - 0s - loss: 202.5693
Epoch 6/100
 - 0s - loss: 170.1475
Epoch 7/100
 - 0s - loss: 144.8341
Epoch 8/100
 - 0s - loss: 138.4752
Epoch 9/100
 - 0s - loss: 134.0544
Epoch 10/100
 - 0s - loss: 111.3680
Epoch 11/100
 - 0s - loss: 79.8450
Epoch 12/100
 - 0s - loss: 66.7497
Epoch 13/100
 - 0s - loss: 57.2966
Epoch 14/100
 - 0s - loss: 51.3331
Epoch 15/100
 - 0s - loss: 46.2316
Epoch 16/100
 - 0s - loss: 44.0735
Epoch 17/100
 - 0s - loss: 41.5919
Epoch 18/100
 - 0s - loss: 40.2799
Epoch 19/100
 - 0s - loss: 38.8085
Epoch 20/100
 - 0s - loss: 37.7297
Epoch 21/100
 - 0s - loss: 36.8116
Epoch 22/100
 - 0s - loss: 35.4354
Epoch 23/100
 - 0s - loss: 35.0248
Epoch 24/100
 - 0s - loss: 33.7619
Epoch 25/100
 - 0s - loss: 32.9706
Epoch 26/100
 - 0s - loss: 32.7900
Epoch 27/100
 - 0s - loss: 32.4153
Epoch 28/100
 - 0s - loss: 30.9434
Epoch 29/100
 -

The code below sets up a neural netowrk and reads the data(for predictions), but it does not clear the model directory or fit the neural network. THe weights from the previous fit are used.

Now we reload the network and perform another prediction. The RMSE should match the previous one exactly if the neural network was really saved and reloaded.

In [6]:
from keras.models import load_model
model2 = load_model(os.path.join(save_path, 'network.h5'))
pred = model2.predict(x)

#Measure RMSE error. RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred, y))
print(f'After load score (RMSE) : {score}')

After load score (RMSE) : 3.7152537509761543


### Thus the scores are the same, this proves it